<a href="https://colab.research.google.com/github/sknctk/osumapper/blob/dev%2Fsknc/osumapper_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## osumapper: create osu! map using Tensorflow and Colab

For mappers who don't know how this colaboratory thing works:
- Press Ctrl+Enter in code blocks to run them one by one
- It will ask you to upload .osu file and audio.mp3 after the third block of code
- .osu file needs to have correct timing (you can use [statementreply](https://osu.ppy.sh/users/126198)'s TimingAnlyz tool)
- After uploading them, wait for a few minutes until download pops

Github: https://github.com/kotritrona/osumapper

reqirements
librosa==0.8.0
numpy==1.22.3
matplotlib==3.2.2
tensorflow>=2.13.1
scikit_learn==1.1.3


### Step 1: Installation

First of all, check the Notebook Settings under Edit tab.<br>
Activate GPU to make the training faster.

Then, clone the git repository and install dependencies.

In [ ]:
%cd /content/
!git clone https://github.com/sknctk/osumapper.git


/content
Cloning into 'osumapper'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 181 (delta 6), reused 16 (delta 2), pack-reused 159 (from 1)
Receiving objects: 100% (181/181), 42.54 MiB | 1.85 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
%cd osumapper/v7.0
!apt install -y ffmpeg
!apt install -y nodejs
!cp requirements_colab.txt requirements.txt
!cp package_colab.json package.json


/content/osumapper/v7.0
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nodejs is already the newest version (18.20.5-1nodesource1).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [ ]:
!sudo apt-get remove -y nodejs libnode72
!sudo apt-get autoremove -y
!sudo apt-get clean
!curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
!sudo apt-get install -y nodejs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libc-ares2 libjs-highlight.js nodejs-doc
Use 'sudo apt autoremove' to remove them.
The following packages will be REMOVED:
  libnode72 nodejs
0 upgraded, 0 newly installed, 2 to remove and 49 not upgraded.
After this operation, 42.1 MB disk space will be freed.
(Reading database ... 124001 files and directories currently installed.)
Removing nodejs (12.22.9~dfsg-1ubuntu3.6) ...
Removing libnode72:amd64 (12.22.9~dfsg-1ubuntu3.6) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libu

In [ ]:
!pip install -r requirements.txt
!npm install

⠙⠹⠸⠼
up to date, audited 6 packages in 691ms
⠼
found 0 vulnerabilities
⠼

### Step 2: Choose a pre-trained model
Set the select_model variable to one of:
- "default": default model
- "sota": model trained with Sota Fujimori music (☆>5.0)
- "vtuber": model trained with VTuber music (☆4.0-5.3)
- "inst": model trained with rhythm game instrumental music (☆3.5-6.5)
- "tvsize": model trained with TV Size maps (☆3.5-5.0 BPM140-190)
- "hard": model trained with Hard difficulties (☆<3.5 BPM140-190)
- "normal": model trained with Normal difficulties (☆<2.7 BPM140-190)
- "lowbpm": model trained with low BPM music (☆3-4.5 BPM<140)
- "taiko": taiko mode model, experimental (☆3-6)
- "catch": catch mode model, experimental (☆3-6)

In [ ]:
from setup_colab import *

select_model = "catch"

model_params = load_pretrained_model(select_model);

### Step 3: Upload map and music file<br>
Map file = .osu file with correct timing<br>
Music file = the mp3 file in the osu folder


In [ ]:
from google.colab import files
print("Please upload the map file:")
mapfile_upload = files.upload()
for fn in mapfile_upload.keys():
  uploaded_osu_name = fn
  print('Uploaded map file: "{name}" {length} bytes'.format(name=fn, length=len(mapfile_upload[fn])))
print("Please upload the music file:")
music_upload = files.upload()
for fn in music_upload.keys():
  print('Uploaded music file: "{name}" {length} bytes'.format(name=fn, length=len(music_upload[fn])))


Please upload the map file:


Saving Camellia - Operation Zenithfall (Mir) [Ai] (2).osu to Camellia - Operation Zenithfall (Mir) [Ai] (2).osu
Uploaded map file: "Camellia - Operation Zenithfall (Mir) [Ai] (2).osu" 164046 bytes
Please upload the music file:


Saving audio.mp3 to audio.mp3
Uploaded music file: "audio.mp3" 14230464 bytes


### Step 4: Read the map and convert to python readable format



In [ ]:
from act_newmap_prep import *

step4_read_new_map(uploaded_osu_name);


### Step 5: Use model to calculate map rhythm

Parameters:

"note_density": how many notes will be placed on the timeline, range (0, 1).<br>
"slider_favor": how the model favors sliders against circles, range (-1.1, 1.1).<br>
"dist_multiplier": the distance snap. range (0, +∞). Of course 0/+∞ are not advisable.<br>
"divisor_favor": how the model favors notes to be on each divisor starting from a beat (white, blue, red, blue), range (-1, 1) each.<br>
"slider_max_ticks": the max amount of time a slider can slide, range (1, +∞).

In [ ]:
from act_rhythm_calc import *

model = step5_load_model(model_file=model_params["rhythm_model"]);
npz = step5_load_npz();

# Or set the parameters here...
params = step5_set_params(dist_multiplier=10, note_density=0.45, slider_favor=0.6, divisor_favor=[0.4] * 4, slider_max_ticks=8);

predictions = step5_predict_notes(model, npz, params);

converted = step5_convert_sliders(predictions, params);

step5_save_predictions(converted);


11/11 [==============================] - 0s 11ms/step
2425 notes predicted.


## Step 6: Map flow generator

Generate the final map using a Generative Adversarial Network (GAN).

Parameters:


- note_distance_basis: the baseline for distance snap between notes
- max_ticks_for_ds: max number of time ticks (each 1/4) that it uses the distance snap
- next_from_slider_end: use slider end instead of slider head for calculating distance
- box_loss_border, box_loss_value: it's like a barrier on the map edges that bounces off the circles
- divisor, note_group_size: don't change unless you're using a special model built for it
- good_epoch, max_epoch: controls the training time. less time makes it faster but risks less quality
- g_\*, c_\*: hyperparameters used by GAN. No one knows how they work but they mysterically affect the result

In [ ]:
from act_gan import *;

GAN_PARAMS = model_params["gan"]
# Or manually set the parameters...
# GAN_PARAMS = {
#     "divisor" : 4,
#     "good_epoch" : 16,
#     "max_epoch" : 36,
#     "note_group_size" : 10,
#     "g_epochs" : 1,
#     "c_epochs" : 1,
#     "g_batch" : 50,
#     "g_input_size" : 50,
#     "c_true_batch" : 140,
#     "c_false_batch" : 5,
#     "c_randfalse_batch" : 5,
#     "note_distance_basis" : 200,
#     "next_from_slider_end" : False,
#     "max_ticks_for_ds" : 1,
#     "box_loss_border" : 0.1,
#     "box_loss_value" : 0.4,
#     "box_loss_weight" : 1
# };
!nvidia-smi
step6_set_gan_params(GAN_PARAMS);
osu_a, data = step6_run_all(flow_dataset_npz=model_params["flow_dataset"]);

Streaming output truncated to the last 5000 lines.
Group 113, Epoch 15: G loss: 0.8772109746932983 vs. C loss: 0.03127727285027504
1/1 [==============================] - 0s 21ms/step
Group 113, Epoch 16: G loss: 1.0031658411026 vs. C loss: 0.045260269194841385
1/1 [==============================] - 0s 21ms/step
Group 113, Epoch 17: G loss: 0.7733617424964905 vs. C loss: 0.08864375948905945
1/1 [==============================] - 0s 20ms/step
Group 113, Epoch 18: G loss: 0.6504237651824951 vs. C loss: 0.09212300181388855
1/1 [==============================] - 0s 21ms/step
Group 113, Epoch 19: G loss: 0.9334457516670227 vs. C loss: 0.06115327775478363
1/1 [==============================] - 0s 18ms/step
Group 113, Epoch 20: G loss: 0.9536682963371277 vs. C loss: 0.03395618498325348
1/1 [==============================] - 0s 19ms/step
Group 113, Epoch 21: G loss: 0.9559375047683716 vs. C loss: 0.03760339692234993
1/1 [==============================] - 0s 20ms/step
Group 113, Epoch 22: G loss

### Since the generation will take a while...

we can appreciate a nice picture of Cute Sophie!!

<img src="https://i.imgur.com/Ko2wogO.jpg" />



Do a little modding to the map.

Parameters:

- stream_regularizer: fix bad streams. integer for modes (0,1,2,3,4) 0=inactive
- slider_mirror: mirror slider ends if they go outside map area. (0,1) 0=inactive 1=active

In [ ]:
from act_modding import *


modding_params = {
    "stream_regularizer" : 1,
    "slider_mirror" : 1
}

osu_a, data = step7_modding(osu_a, data, modding_params);

Finally, save the data into an .osu file!

In [ ]:
from google.colab import files
from act_final import *
from act_taiko_hitsounds import *

if select_model == "taiko":
    taiko_hitsounds_params = step8_taiko_hitsounds_set_params(divisor=4, metronome_count=4)
    hitsounds = step8_apply_taiko_hitsounds(osu_a, data, hs_dataset=model_params["hs_dataset"], params=taiko_hitsounds_params)
    saved_osu_name = step8_save_osu_file(osu_a, data, hitsounds=hitsounds)
else:
    saved_osu_name = step8_save_osu_file(osu_a, data);

files.download(saved_osu_name)

success! built osu map from json data.
elapsed time: 0.296 s

finished on: 2024-12-04 15:48:56.826456


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# clean up if you want to make another map!
# colab_clean_up(uploaded_osu_name)

That's it! Now you can try out the AI-created map in osu!.

For bug reports and feedbacks either report it on github or use discord: <br>
[https://discord.com/invite/npmSy7K](https://discord.com/invite/npmSy7K)